## HapMap 3 pipline for QuAP

In [1]:
files = [f'hapmap3/hapmap3_r1_b36_fwd.ASW.qc.poly.recode', 
         f'hapmap3/hapmap3_r1_b36_fwd.CEU.qc.poly.recode', 
         f'hapmap3/hapmap3_r1_b36_fwd.YRI.qc.poly.recode']

In [2]:
# deleting of close relatives
# prepare .remove file
def remove_relatives(filename):
    f = open(f'{filename}.ped')
    out = open(f'{filename}.remove', 'w')
    n = 0
    c = 0
    for line in f:
        entry = list(line.split())
        if (entry[2] != '0') or (entry[3] != '0'):
            c += 1
            out.write(entry[0]+'\t'+entry[1]+'\n')
        n+=1
    c/n
    f.close()
    out.close()

In [3]:
# removing relatives from data
for file in files:
    remove_relatives(file)
    !./plink --file {file} --recode 12 --out {file}.filtered --remove {file}.remove --chr 1-22

PLINK v1.90b6.26 64-bit (2 Apr 2022)           www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap3/hapmap3_r1_b36_fwd.ASW.qc.poly.recode.filtered.log.
Options in effect:
  --chr 1-22
  --file hapmap3/hapmap3_r1_b36_fwd.ASW.qc.poly.recode
  --out hapmap3/hapmap3_r1_b36_fwd.ASW.qc.poly.recode.filtered
  --recode 12
  --remove hapmap3/hapmap3_r1_b36_fwd.ASW.qc.poly.recode.remove

8192 MB RAM detected; reserving 4096 MB for main workspace.
.ped scan complete (for binary autoconversion).82930323335363839404243454647495052535456575960616364666769707173747677788081838485878890919294959798100%
Performing single-pass .bed write (1482375 variants, 71 people).
--file: hapmap3/hapmap3_r1_b36_fwd.ASW.qc.poly.recode.filtered-temporary.bed +49505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394%
hapmap3/hapmap3_r1_b36_fwd.ASW.qc.poly.recode.filtered-temporary.bim +
hapmap3/hapmap3_r1_b3

In [4]:
# merge 3 population into one file
!./plink --file {files[0]}.filtered --recode --out ASW_CEU  --merge {files[1]}.filtered # --indiv-sort 0
!./plink --file ASW_CEU --recode --out ASW_CEU_YRI --merge {files[2]}.filtered # --indiv-sort 0

PLINK v1.90b6.26 64-bit (2 Apr 2022)           www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ASW_CEU.log.
Options in effect:
  --file hapmap3/hapmap3_r1_b36_fwd.ASW.qc.poly.recode.filtered
  --merge hapmap3/hapmap3_r1_b36_fwd.CEU.qc.poly.recode.filtered
  --out ASW_CEU
  --recode

8192 MB RAM detected; reserving 4096 MB for main workspace.
.ped scan complete (for binary autoconversion).0424547505254575961646669717376788083858890929597100%
Performing single-pass .bed write (1482375 variants, 42 people).
--file: ASW_CEU-temporary.bed + ASW_CEU-temporary.bim + ASW_CEU-temporary.fam849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394%
written.
42 people loaded from ASW_CEU-temporary.fam.
109 people to be merged from
hapmap3/hapmap3_r1_b36_fwd.CEU.qc.poly.recode.filtered.ped.
Of these, 109 are new, while 0 are present in the base dataset.
1482375 markers loaded from ASW_CEU-t

In [5]:
# filter for LD (from ADMIXTURE manual)
!./plink --file ASW_CEU_YRI --indep-pairwise 50 10 0.1
!./plink --file ASW_CEU_YRI --extract plink.prune.in --make-bed --out ASW_CEU_YRI_all
!./plink --recode --bfile ASW_CEU_YRI_all --out ASW_CEU_YRI_all

PLINK v1.90b6.26 64-bit (2 Apr 2022)           www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to plink.log.
Options in effect:
  --file ASW_CEU_YRI
  --indep-pairwise 50 10 0.1

8192 MB RAM detected; reserving 4096 MB for main workspace.
.ped scan complete (for binary autoconversion).415151516161617171818181919202020212122222223232324242525252626272727282828292930303031313232323333333434353535363637373738383839394040404141424242434344444445454546464747474848494949505050515152525253535454545555555656575757585859595960606161616262626363646464656566666667676768686969697070717171727272737374747475757676767777777878797979808081818182828383838484848585868686878788888889898990909191919292939393949494959596969697979898989999100%
Performing single-pass .bed write (1538208 variants, 259 people).
--file: plink-temporary.bed + plink-temporary.bim + plink-temporary.fam546474849505152535455565758596061626364656667686970717273747

In [6]:
# save choromosomes to separate files
for chr_i in range(1, 23):
    !./plink --bfile ASW_CEU_YRI_all --chr {chr_i} --make-bed --out ASW_CEU_YRI_{chr_i}

PLINK v1.90b6.26 64-bit (2 Apr 2022)           www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ASW_CEU_YRI_1.log.
Options in effect:
  --bfile ASW_CEU_YRI_all
  --chr 1
  --make-bed
  --out ASW_CEU_YRI_1

8192 MB RAM detected; reserving 4096 MB for main workspace.
15391 out of 187453 variants loaded from .bim file.
259 people (127 males, 132 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 259 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.888172.
15391 variants and 259 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ASW_CEU_YRI_1.bed + ASW_CEU_YRI_1.bim + ASW_CEU_YRI_1.fam ...
1011121314151617181

9539 out of 187453 variants loaded from .bim file.
259 people (127 males, 132 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 259 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.900579.
9539 variants and 259 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ASW_CEU_YRI_8.bed + ASW_CEU_YRI_8.bim + ASW_CEU_YRI_8.fam ...
101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.26 64-bit (2 Apr 2022)           www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ASW_C

PLINK v1.90b6.26 64-bit (2 Apr 2022)           www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ASW_CEU_YRI_16.log.
Options in effect:
  --bfile ASW_CEU_YRI_all
  --chr 16
  --make-bed
  --out ASW_CEU_YRI_16

8192 MB RAM detected; reserving 4096 MB for main workspace.
6643 out of 187453 variants loaded from .bim file.
259 people (127 males, 132 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 259 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.896497.
6643 variants and 259 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ASW_CEU_YRI_16.bed + ASW_CEU_YRI_16.bim + ASW_CEU_YRI_16.fam ...
101112131415161

In [7]:
# prepare ADMIXTURE .pop file for --supervised mode.

prefs = ["ASW", "CEU", "YRI"]
ind = [[], [], []]
for i, file in enumerate(files):
    f = open(f'{file}.filtered.ped')
    for line in f:
        ind[i].append(line.split()[0])
        
out = open(f'ASW_CEU_YRI_all.pop', 'w')
f = open(f'ASW_CEU_YRI_all.fam')
for line in f:
    fam = line.split()[0]
    if fam in ind[0]:
        out.write('-' + '\n')
    if fam in ind[1]:
        out.write(prefs[1] + '\n')
    if fam in ind[2]:
        out.write(prefs[2] + '\n')
out.close()
f.close()